In [ ]:
from common import *

def load_group(filename):
    with open(filename, 'r') as file:
        data = file.read()
        # _, _, data = data.partition('\n')
    try:
        lua = lupa.LuaRuntime(unpack_returned_tuples=True)
        lua.execute(data)
        g = lua.globals()
        d = dict(g)
        gadgets = g['gadgets']
        return table_to_dict(gadgets)
    except lupa.LuaError as e:
        print(filename, e)
path = '/home/luke/code/GenshinImpact/Grasscutter/resources/Scripts/Scene/'
# filenames = [s for s in os.listdir(path) if s.endswith('.lua')]

In [ ]:
lua_gadgets = {}
errors = []
path = '/home/luke/code/GenshinImpact/Grasscutter/resources/Scripts/Scene/'
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.lua'):
            full = os.path.join(root, file)
            gadgets = load_group(full)
            key = full[len(path):]
            if gadgets is not None:
                lua_gadgets[key] = gadgets
            else:
                errors.append(key)
lua_gadgets['errors'] = errors
with open('gadgets.lua.json', 'w', encoding='utf-8', newline='\n') as file:
    file.write(encode_json(lua_gadgets))

In [ ]:
lua_gadgets_decoded = load_json('gadgets.lua.json', True)
lua_gadgets_flat = {}
for key, gadgets in lua_gadgets_decoded.items():
    if isinstance(gadgets, list):
        lua_gadgets_flat[key] = [flatten_json(g, flatten_xyz=True, flatten_lists=False) for g in gadgets]
    elif isinstance(gadgets, dict):
        lua_gadgets_flat[key] = flatten_json(gadgets, flatten_xyz=True, flatten_lists=False)
    else:
        lua_gadgets_flat[key] = gadgets
with open('gadgets.flat.lua.json', 'w', encoding='utf-8', newline='\n') as file:
    file.write(encode_json(lua_gadgets_flat))